code in functions
#super function not working

In [82]:


zipfile_ou = '../data/anonymisedData.zip'
prediction_point = 200

import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt




In [83]:


def load_data(zip_file_path):
    '''Loads the data from the Open University Learning Analytics dataset zip file.'''
    
    with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
        registrations = pd.read_csv(zip_file.open('studentRegistration.csv'))
        courses = pd.read_csv(zip_file.open('courses.csv'))
        students = pd.read_csv(zip_file.open('studentInfo.csv'))
        student_vle = pd.read_csv(zip_file.open('studentVle.csv'))
        vle = pd.read_csv(zip_file.open('vle.csv'))
        student_assessments = pd.read_csv(zip_file.open('studentAssessment.csv'))
        assessments = pd.read_csv(zip_file.open('assessments.csv'))
    
    return registrations, courses, students, student_vle, vle, student_assessments, assessments






In [84]:
registrations, courses, students, student_vle, vle, student_assessments, assessments = load_data(zipfile_ou)

In [86]:
#assessments.info()
#courses.info()

In [87]:
def student_data(students, courses, registrations, prediction_point=None):
    '''Returns a dataframe of student data based on `students`: 

    prediction_point is an integer representing the number of days from the start of the course for which you want to predict the outcome.

    default is max(student_regist['module_presentation_length']) - that is, the last da of the lengthiest course

    - merge students and courses
    - merge registrations
    - drop missing value rows (date_registration, imd_band)
    - drop students who unregistered before registering
    - fills missing date_unregistration with module_presentation_length - that is, assumes they completed the course
    - removes students who withdrew or failed before prediction point
    '''

    # merge students and courses
    student_regist = pd.merge(students, courses, on=['code_module', 'code_presentation'], validate='many_to_one')

    # merge registrations
    student_regist = pd.merge(student_regist, registrations, on=['code_module', 'code_presentation', 'id_student'], how = 'left', validate='1:1')

    #prediction_point default
    if prediction_point is None:
        prediction_point = max(student_regist['module_presentation_length']) 

    # drop missing value rows (date_registration, imd_band)
    student_regist.dropna(subset=['date_registration', 'imd_band'], inplace=True)

    # prediction point must be less than course length, integer, and greater than 0
    if not isinstance(prediction_point, int) or prediction_point <= 0 or prediction_point >= max(student_regist['module_presentation_length']):
        print("Error: Invalid prediction point. \n\nPlease provide an integer value greater than 0 and less than the maximum course length. \n\nThis is the number of days from the start of the course for which you want to predict the outcome.")
    else:
        # withdrawn or failed before prediction point - remove
        withdrawn_fail_condition = (student_regist['final_result'].isin(['Withdrawn', 'Fail'])) & (student_regist['date_unregistration'] <= prediction_point)
        student_regist.loc[withdrawn_fail_condition, 'status'] = 'remove_outcome_known'
        # if unregister after prediction point - keep
        unregister_after_condition = student_regist['date_unregistration'] > prediction_point
        student_regist.loc[unregister_after_condition, 'status'] = 'keep'
        # if no unregistration date - keep
        no_unregistration_condition = student_regist['date_unregistration'].isna()
        student_regist.loc[no_unregistration_condition, 'status'] = 'keep'
        # default case
        student_regist.loc[~(withdrawn_fail_condition | unregister_after_condition | no_unregistration_condition), 'status'] = 'query'

    # rows which need investigation
    query_rows = student_regist[student_regist['status'] == 'query'] | student_regist[student_regist['status'].isna()]

    # print rows which need investigation
    if not query_rows.empty:
        print("The following rows need investigation.  They are excluded from the following analysis: \n")
        print(query_rows)
        student_regist = student_regist[~student_regist.isin(query_rows)].dropna()

    # replace missing date_unreg with module_presentation_length
    student_regist['date_unregistration'] = student_regist['date_unregistration'].fillna(student_regist['module_presentation_length'])

    # drop students who unregistered before registering
    student_regist = student_regist[student_regist['date_unregistration'] >= student_regist['date_registration']]

    # remove rows from final student df which are not needed
    model_final = student_regist[student_regist['status'] != 'remove_outcome_known']
   

    return model_final



In [88]:
model_final = student_data(students, courses, registrations, prediction_point)

In [89]:
def add_assessments(student_assessments, assessments, courses, model_final, prediction_point=None):
    '''Returns updated model_final dataframe with student assessment data added:

    prediction_point is an integer representing the number of days from the start of the course for which you want to predict the outcome.

    default is max(student_regist['module_presentation_length']) - that is, the last da of the lengthiest course

    - merge 'assessments' and 'courses' and 'student_assessments'
    - populate missing 'date' values with course final week - as suggested in literature
    - remove students from 'model_final' who have no score for an assessment
    - remove students with odd assessment dates (before registration, after unregistration, or very far into future)
    - remove students who withdrew or failed before prediction point
    - calculate new features - average score, submission date distance, proportion of assessments submitted
    '''
    # merge 'assessments' and 'courses' on 'code_module' and 'code_presentation'
    course_assess = pd.merge(assessments,courses, on=['code_module', 'code_presentation'], how='left')

    # fill in the missing 'date' values with course final week (as per literature)
    value_to_fill = course_assess['module_presentation_length'] - 3
    course_assess['date'] = course_assess['date'].fillna(value_to_fill)

    # merge student_assessments with course_assess
    stu_assess = pd.merge(student_assessments, course_assess, on=['id_assessment'], how='left')

    # drop students who have no score for an assessment
    missing_score_rows = stu_assess[stu_assess['score'].isna()]

    if not missing_score_rows.empty:
        print("\n\nThe following students have missing 'scores'. They are excluded from the analysis: \n")
        print(missing_score_rows)

        # student ids with missing score
        unique_ids_missing = missing_score_rows['id_student'].unique()

        # remove students with missing score from model_final
        model_final = model_final[~model_final['id_student'].isin(unique_ids_missing)]

        # drop rows with missing score
        stu_assess.dropna(subset=['score'], inplace=True)

    # remove students with negative or extreme date_submitted values
    max_module_length = stu_assess['module_presentation_length'].max()
    greater_than_max_length = stu_assess[stu_assess['date_submitted'] > max_module_length]['id_student'].unique()
    less_than_zero = stu_assess[stu_assess['date_submitted'] < 0]['id_student'].unique()
    students_to_remove = set(greater_than_max_length) | set(less_than_zero)
    model_final = model_final[~model_final['id_student'].isin(students_to_remove)]
    stu_assess = stu_assess[~stu_assess['id_student'].isin(students_to_remove)]

    # reduce data by prediction point
    model_student_assess = stu_assess[stu_assess['date'] <= prediction_point]

    # get expected assessment details
    model_course_assess = course_assess[course_assess['date'] <= prediction_point]
    expected_ass = model_course_assess.groupby(['code_module', 'code_presentation'])['id_assessment'].count().reset_index()
    expected_ass = expected_ass.rename(columns={'id_assessment': 'exp_sub_count'})
    date_sum = model_course_assess.groupby(['code_module', 'code_presentation'])['date'].sum().reset_index()
    expected_ass = expected_ass.merge(date_sum, on=['code_module', 'code_presentation'], how='left')
    expected_ass = expected_ass.rename(columns={'date': 'exp_sub_date_sum'})

    # summarise students' assessments by module_presentation
    student_assessment_summary = model_student_assess.groupby(['id_student', 'code_module', 'code_presentation']).agg(
        count_id_assessment=('id_assessment', 'count'),
        sum_score=('score', 'sum'),
        sum_date=('date', 'sum')
    ).reset_index()

    # merge student_assessment_summary and expected_ass on code_module and code_presentation
    merged_assess_summary = student_assessment_summary.merge(expected_ass, on=['code_module', 'code_presentation'])

    # calculate the new features
    merged_assess_summary['prop_submissions'] = merged_assess_summary['count_id_assessment'] / merged_assess_summary['exp_sub_count']
    merged_assess_summary['avg_score'] = merged_assess_summary['sum_score'] / merged_assess_summary['exp_sub_count']
    merged_assess_summary['submission_distance'] = merged_assess_summary['exp_sub_date_sum'] - merged_assess_summary['sum_date']

    # merge with 'model_final'
    model_final = model_final.merge(merged_assess_summary, on=['id_student', 'code_module', 'code_presentation'], how='inner')

    return model_final

In [90]:
model_final = add_assessments(student_assessments, assessments, courses, model_final, prediction_point)



The following students have missing 'scores'. They are excluded from the analysis: 

        id_assessment  id_student  date_submitted  is_banked  score  \
215              1752      721259              22          0    NaN   
937              1754      260355             127          0    NaN   
2364             1760     2606802             180          0    NaN   
3358            14984      186780              77          0    NaN   
3914            14984      531205              26          0    NaN   
...               ...         ...             ...        ...    ...   
148929          34903      582670             241          0    NaN   
159251          37415      610738              87          0    NaN   
166390          37427      631786             221          0    NaN   
169725          37435      648110              62          0    NaN   
170103          37435      480914              49          0    NaN   

       code_module code_presentation assessment_type   date 

In [91]:
#model_final.count()

code_module                   20073
code_presentation             20073
id_student                    20073
gender                        20073
region                        20073
highest_education             20073
imd_band                      20073
age_band                      20073
num_of_prev_attempts          20073
studied_credits               20073
disability                    20073
final_result                  20073
module_presentation_length    20073
date_registration             20073
date_unregistration           20073
status                        20073
count_id_assessment           20073
sum_score                     20073
sum_date                      20073
exp_sub_count                 20073
exp_sub_date_sum              20073
prop_submissions              20073
avg_score                     20073
submission_distance           20073
dtype: int64

In [92]:
def add_vle_data(model_final, student_vle, vle, courses, prediction_point=None):
    '''Returns updated model_final dataframe with vle data:

    prediction_point is an integer representing the number of days from the start of the course for which you want to predict the outcome.

    default is max(student_regist['module_presentation_length']) - that is, the last da of the lengthiest course

    - merge 'vle' and 'courses' and 'student_vle'
    - remove columns (week_from, week_to)
    - filter the rows where 'date' <= 'prediction_point'
    - create new features: vle_activity_count, student vle type count, student total clicks, student days active (engaged)
    
    '''
    # merge 'vle' and 'courses' on 'code_module' and 'code_presentation'
    course_vle = vle.merge(courses, on=['code_module', 'code_presentation'], how='left').drop(['week_from', 'week_to'], axis=1)

    # merge vle with student_vle
    all_stu_vle = pd.merge(student_vle, course_vle, on=['id_site', 'code_module', 'code_presentation'], how='left')

    # filter the rows where 'date' <= 'prediction_point'
    all_stu_vle = all_stu_vle[all_stu_vle['date'] <= prediction_point]

    # filter the rows where 'date' is greater than 'module_presentation_length'
    vle_after_done = all_stu_vle[all_stu_vle['date'] > all_stu_vle['module_presentation_length']]

    if not vle_after_done.empty:
        print("The following rows need investigation. They are excluded from the following analysis: \n")
        print(vle_after_done)

        # match rows based on 'code_module', 'code_presentation', and 'id_student'
        matching_rows = model_final[model_final[['code_module', 'code_presentation', 'id_student']].isin(vle_after_done).all(axis=1)]

        # remove the matching rows from 'model_final'
        model_final = model_final[~model_final.index.isin(matching_rows.index)]

    # aggregations for each column
    aggregations = {
        'id_site': 'count',
        'activity_type': 'nunique',
        'sum_click': 'sum',
        'date': lambda x: x.nunique()
    }

    # group and apply the aggregations
    grouped_stu_vle = all_stu_vle.groupby(['code_module', 'code_presentation', 'id_student']).agg(aggregations).reset_index()

    # rename the columns
    grouped_stu_vle.rename(columns={
        'id_site': 'stu_activity_count',
        'activity_type': 'stu_activity_type_count',
        'sum_click': 'stu_total_clicks',
        'date': 'stu_days_active'
    }, inplace=True)

    # number of vle types per module_presentation
    group_vle = vle.groupby(['code_module', 'code_presentation'])['activity_type'].nunique().reset_index()
    group_vle.rename(columns={'activity_type': 'mod_pres_vle_type_count'}, inplace=True)

    # merge 'grouped_stu_vle' and 'group_vle' on 'code_module' and 'code_presentation'
    merged_vle_summary = grouped_stu_vle.merge(group_vle, on=['code_module', 'code_presentation'], how='left')

    # merge with 'model_final'
    model_final = model_final.merge(merged_vle_summary, on=['id_student', 'code_module', 'code_presentation'], how='inner')

    return model_final


In [93]:
model_final = add_vle_data(model_final, student_vle, vle, courses, prediction_point)

In [94]:
model_final.count()

code_module                   20063
code_presentation             20063
id_student                    20063
gender                        20063
region                        20063
highest_education             20063
imd_band                      20063
age_band                      20063
num_of_prev_attempts          20063
studied_credits               20063
disability                    20063
final_result                  20063
module_presentation_length    20063
date_registration             20063
date_unregistration           20063
status                        20063
count_id_assessment           20063
sum_score                     20063
sum_date                      20063
exp_sub_count                 20063
exp_sub_date_sum              20063
prop_submissions              20063
avg_score                     20063
submission_distance           20063
stu_activity_count            20063
stu_activity_type_count       20063
stu_total_clicks              20063
stu_days_active             

In [95]:
def tidy_up_model_final(model_final):
    '''Returns updated model_final dataframe:
    - deleted unnecessary columns
    - reordered columns
    - added subject, year, month columns
    
    '''
    # new columns for module_presentation - subject, year, month
    model_final['year'] = model_final['code_presentation'].str[:4].astype(int)
    model_final['month'] = model_final['code_presentation'].str[-1].map({'J': 'Oct', 'B': 'Feb'})

    # module subject mapping
    code_module_mapping = {
        'AAA': 'SocSci',
        'BBB': 'SocSci',
        'GGG': 'SocSci',
        'CCC': 'Stem',
        'DDD': 'Stem',
        'EEE': 'Stem',
        'FFF': 'Stem'
    }
    model_final['subject'] = model_final['code_module'].map(code_module_mapping)

    # rename 'module_presentation_length' to 'course_length'
    model_final.rename(columns={'module_presentation_length': 'course_length'}, inplace=True)

    # combine 'code_module', 'code_presentation', and 'id_student' into 'mod_pres_stu'
    model_final['mod_pres_stu'] = model_final['code_module'] + '-' + model_final['code_presentation'] + '-' + model_final['id_student'].astype(str)

    # drop the separate columns 'code_module', 'code_presentation', and 'id_student'
    model_final.drop(columns=['code_module', 'code_presentation', 'id_student'], inplace=True)

    # move 'final_result' to the last column
    final_result_column = model_final.pop('final_result')
    model_final['final_result'] = final_result_column

    # insert 'mod_pres_stu' as the first column
    mod_pres_stu_column = model_final.pop('mod_pres_stu')
    model_final.insert(0, 'mod_pres_stu', mod_pres_stu_column)

    # drop columns: 'count_id_assessment', 'sum_score', 'sum_date', 'exp_sub_count', 'exp_sub_date_sum'
    model_final.drop(columns=['count_id_assessment', 'sum_score', 'sum_date', 'exp_sub_count', 'exp_sub_date_sum'], inplace=True)

    return model_final


In [96]:
model_final = tidy_up_model_final(model_final)

In [97]:
model_final.count()

mod_pres_stu               20063
gender                     20063
region                     20063
highest_education          20063
imd_band                   20063
age_band                   20063
num_of_prev_attempts       20063
studied_credits            20063
disability                 20063
course_length              20063
date_registration          20063
date_unregistration        20063
status                     20063
prop_submissions           20063
avg_score                  20063
submission_distance        20063
stu_activity_count         20063
stu_activity_type_count    20063
stu_total_clicks           20063
stu_days_active            20063
mod_pres_vle_type_count    20063
year                       20063
month                      20063
subject                    20063
final_result               20063
dtype: int64

In [101]:
model_final.head()

duplicate_students = model_final[model_final.duplicated(subset='mod_pres_stu', keep=False)]['mod_pres_stu'].unique()

duplicate_students

missing_values = model_final.isnull().sum()
print("Missing values:\n", missing_values, "\n")

Missing values:
 mod_pres_stu               0
gender                     0
region                     0
highest_education          0
imd_band                   0
age_band                   0
num_of_prev_attempts       0
studied_credits            0
disability                 0
course_length              0
date_registration          0
date_unregistration        0
status                     0
prop_submissions           0
avg_score                  0
submission_distance        0
stu_activity_count         0
stu_activity_type_count    0
stu_total_clicks           0
stu_days_active            0
mod_pres_vle_type_count    0
year                       0
month                      0
subject                    0
final_result               0
dtype: int64 



In [80]:
# not working

def process_data(zip_file, prediction_point):
    # Load data from the zip file
    registrations, courses, students, student_vle, vle, student_assessments, assessments = load_data(zip_file)

    # Process student data
    model_final = student_data(students, courses, registrations, prediction_point)

    # Add assessments to model_final
    model_final = add_assessments(model_final, student_assessments, assessments, courses, prediction_point)

    # Add VLE data to model_final
    model_final = add_vle_data(model_final, student_vle, vle, courses, prediction_point)

    # Tidy up model_final
    model_final = tidy_up_model_final(model_final)

    return model_final




In [81]:
# not working

model_final = process_data(zipfile_ou, prediction_point)

KeyError: 'code_module'

add default for prediction point, ie. if none